In [1]:
# !pip install accelerate

In [2]:
# !pip install sentence_transformers

In [3]:
# import json
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# from sentence_transformers import SentenceTransformer
# from sklearn.metrics.pairwise import cosine_similarity
# import torch

# # # Load the TLQA datasets
# # with open('/content/train_TLQA.json', 'r') as file:
# #     train_tlqa_data = json.load(file)

# # with open('/content/test_TLQA.json', 'r') as file:
# #     test_tlqa_data = json.load(file)

# # Load the TLQA datasets in kaggle
# with open('/kaggle/input/tlqa-nlp/data/train_TLQA.json', 'r') as file:
#     train_tlqa_data = json.load(file)

# with open('/kaggle/input/tlqa-nlp/data/test_TLQA.json', 'r') as file:
#     test_tlqa_data = json.load(file)
    
# # Extract questions and answers from the training dataset
# train_questions = [item['question'] for item in train_tlqa_data]
# train_answers = [item['answers'] for item in train_tlqa_data]

# test_questions = [item['question'] for item in test_tlqa_data]
# test_answers = [item['answers'] for item in test_tlqa_data]

# # Extract full examples (questions and answers) from the dataset
# train_examples = [f"Question: {item['question']} Answer: {', '.join(item['answers'])}" for item in train_tlqa_data]
# test_examples = [f"Question: {item['question']} Answer: {', '.join(item['answers'])}" for item in test_tlqa_data]

In [4]:
# class KnnSearch:
#     def __init__(self, num_trees=None, emb_dim=None):
#         self.num_trees = num_trees
#         self.emb_dim = emb_dim

#     def get_embeddings_for_data(self, data_ls):
#         model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
#         embeddings = model.encode(data_ls)
#         return embeddings

#     def get_top_n_neighbours(self, sentence, data_emb, transfer_data, k):
#         sent_emb = self.get_embeddings_for_data([sentence])[0]
#         top_questions = []

#         text_sims = cosine_similarity(data_emb, [sent_emb]).tolist()
#         results_sims = zip(range(len(text_sims)), text_sims)
#         sorted_similarities = sorted(results_sims, key=lambda x: x[1], reverse=True)

#         for idx, item in sorted_similarities[:k]:
#             top_questions.append(transfer_data[idx])

#         return top_questions

# knn_search = KnnSearch()
# train_data_embeddings = knn_search.get_embeddings_for_data(train_examples)


In [5]:
# def few_shot_qa(test_item, k, data_embeddings, train_data, model, tokenizer):
#     test_question = test_item['question']

#     top_k_examples = knn_search.get_top_n_neighbours(test_question, data_embeddings, train_data, k)

#     input_text = ""
#     for example in top_k_examples:
#         input_text += f"Example Question: {example['question']} Example Answer: {' '.join(example['answers'])} "

#     input_text += f"Question: {test_question} Answer:"
#     input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
#     print(input_text)
    
#     outputs = model.generate(input_ids)
#     answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     return answer

# # modfiied 
# def few_shot_qa(test_item, k, data_embeddings, train_data, model, tokenizer):
#     test_question = test_item['question']

#     top_k_examples = knn_search.get_top_n_neighbours(test_question, data_embeddings, train_data, k)

#     input_text = ""
#     for idx, example in enumerate(top_k_examples, 1):
#         input_text += f"Example Question {idx}: {example['question']}\n"
#         input_text += f"Example Answer {idx}: {' '.join(example['answers'])}\n"

#     input_text += f"Question: {test_question} Answer:"
#     input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
#     print(input_text)
    
#     outputs = model.generate(input_ids)
#     answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     return answer

In [6]:
# def prepared_question(test_tlqa_data, k, data_embeddings, train_data):
    
#     for test_item in test_tlqa_data:
#         test_question = test_item['question']
#         top_k_examples = knn_search.get_top_n_neighbours(test_question, data_embeddings, train_data, k)
#         input_text = ""
#         for example in top_k_examples:
#             input_text += f"Example Question: {example['question']} Example Answer: {' '.join(example['answers'])} "

#         input_text += f"Question: {test_question} Answer:"
#         test_item['question_with_KNN_example'] = input_text

#     # 将更新后的数据写入新的test_TLQA.json文件
#     output_file_path = f'/kaggle/working/test_TLQA_with_KNN_k={k}.json'
#     with open(output_file_path, 'w') as file:
#         json.dump(test_tlqa_data, file, indent=4)

In [7]:
# for k in [3,5,7,10]:
#     prepared_question(test_tlqa_data, k, train_data_embeddings, train_tlqa_data)

In [8]:
# import json
# import torch
# from sentence_transformers import SentenceTransformer
# from sklearn.metrics.pairwise import cosine_similarity
# import os
# import time

# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# class KnnSearch:
#     def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
#         self.model = SentenceTransformer(model_name)
#         self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#         self.model.to(self.device)

#     def get_embeddings_for_data(self, data_ls):
#         embeddings = self.model.encode(data_ls, batch_size=32, show_progress_bar=True, device=self.device)
#         return embeddings

#     def compute_similarities(self, test_data_emb, train_data_emb):
#         return cosine_similarity(test_data_emb, train_data_emb)

#     def get_top_n_neighbours(self, similarities, transfer_data, k):
#         sorted_indices = similarities.argsort()[::-1][:k]
#         top_questions = [transfer_data[idx] for idx in sorted_indices]
#         return top_questions

# def prepared_question(test_tlqa_data, k, similarities, train_data, knn_search):
#     start_time = time.time()
#     for idx, test_item in enumerate(test_tlqa_data):
#         test_question = test_item['question']
#         top_k_examples = knn_search.get_top_n_neighbours(similarities[idx], train_data, k)
        
#         input_text = "I will give you some examples for Timeline based List-Question Answering, and you need to answer the question, and the format as answers in example which need contain answer and time /n"
#         for i, example in enumerate(top_k_examples, 1):
#             input_text += f"Example  {i}: {example}\n"
    
#         input_text += f"Question: {test_question} Answer:"
#         test_item['question_with_KNN_example'] = input_text

#     end_time = time.time()
#     print(f"Processed all items with k={k} in {end_time - start_time:.2f} seconds.")
#     return test_tlqa_data

# if __name__ == "__main__":
#     start_time = time.time()
#     knn_search = KnnSearch()

#     # 加载数据
#     with open('/kaggle/input/tlqa-nlp/data/train_TLQA.json', 'r') as file:
#         train_tlqa_data = json.load(file)

#     with open('/kaggle/input/tlqa-nlp/data/test_TLQA.json', 'r') as file:
#         test_tlqa_data = json.load(file)

#     # 提取问题和答案
#     train_questions = [item['question'] for item in train_tlqa_data]
#     train_answers = [item['answers'] for item in train_tlqa_data]

#     # 获取训练和测试数据的嵌入
#     print("Computing embeddings...")
#     train_data_embeddings = knn_search.get_embeddings_for_data(train_questions)
#     test_data_embeddings = knn_search.get_embeddings_for_data([item['question'] for item in test_tlqa_data])
#     print(f"Computed embeddings in {time.time() - start_time:.2f} seconds.")

#     # 计算相似度
#     similarities = knn_search.compute_similarities(test_data_embeddings, train_data_embeddings)

#     # 处理不同的 k 值
#     for k in [3, 5, 7, 10]:
#         test_tlqa_data = prepared_question(test_tlqa_data, k, similarities, train_tlqa_data, knn_search)
        
#         output_file_path = f'/kaggle/working/test_TLQA_with_KNN_k={k}_all_information.json'
#         with open(output_file_path, 'w') as file:
#             json.dump(test_tlqa_data, file, indent=4)


In [9]:
# del model
# del input_ids
# del attention_mask
# torch.cuda.empty_cache()

# Inference

In [10]:
import json
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import DataLoader, Dataset

class TLQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        inputs = item['question_with_KNN_example']
        targets = ["; ".join(item['answers'])]  # Join list of answers with a separator
        model_inputs = self.tokenizer(inputs, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt")
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(targets, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt")
        return {
            'input_ids': model_inputs['input_ids'].squeeze(0),
            'attention_mask': model_inputs['attention_mask'].squeeze(0),
            'labels': labels['input_ids'].squeeze(0),
            'question': inputs,
            'answers': item['answers']  # Include the actual answers for comparison
        }

In [11]:
def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    questions = [item['question'] for item in batch]
    answers = [item['answers'] for item in batch]
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels, 'question': questions, 'answers': answers}

In [12]:
# def generate_answers(outputs, tokenizer):
#     decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
#     return [output.split("; ") for output in decoded_outputs]

def generate_answers(outputs, tokenizer):
    decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    # formatted_outputs = [eval(output) if output.startswith("[") else [output] for output in decoded_outputs]
    return decoded_outputs

In [13]:
# Load the Flan-T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-large')
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Using 2 GPUs


DataParallel(
  (module): T5ForConditionalGeneration(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
                (wi

In [14]:
# Load test data
with open('/kaggle/input/tlqa-withknn/TLQA/test_TLQA_with_KNN_k7_all_information_prepromopt.json', 'r') as file:
    test_data = json.load(file)

In [15]:
# Create dataset and dataloader
dataset = TLQADataset(test_data, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, collate_fn=collate_fn)

model.eval()
predictions = []
actual_answers = []

with torch.no_grad():
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model.module.generate(input_ids, attention_mask=attention_mask, max_length=512, num_beams=1, early_stopping=True) if torch.cuda.device_count() > 1 else model.generate(input_ids, attention_mask=attention_mask, max_length=512, num_beams=1, early_stopping=True)
        batch_predictions = generate_answers(outputs, tokenizer)
        predictions.extend(batch_predictions)
        actual_answers.extend(batch['answers'])
        
        #torch.cuda.empty_cache()


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [16]:
with open('predictions-FlanT5-large-k=7.json', 'w') as f:
    json.dump(predictions, f)

In [17]:
# # Perform inference on test set
# def generate_answers(batch):
#     inputs = batch['input_ids'].to(model.device)
#     outputs = model.generate(
#         inputs,
#         max_length=64,
#         num_beams=5,
#         early_stopping=True,
#         repetition_penalty=2.5,
#         length_penalty=1.0,
#         no_repeat_ngram_size=2,
#         num_return_sequences=1  # Generate a single sequence per input
#     )
#     decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#     return [output.split("; ") for output in decoded_outputs]

In [18]:
# # Trainer
# training_args = TrainingArguments(
#     output_dir="./results-FlanT5-base",
#     evaluation_strategy="epoch",
#     per_device_eval_batch_size=1
# )

# trainer = Trainer(
#     model=model,
#     args=training_args
# )

In [19]:
# predictions = []
# for batch in trainer.get_test_dataloader(tokenized_test):
#     batch_predictions = generate_answers(batch)
#     predictions.extend(batch_predictions)